# Deep Dive into LangChain

In [1]:
pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


### Python-dotenv

In [2]:
import os 
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.environ.get('OPENAI_API_KEY')

'sk-proj-dsxvcH7KXpdycO17nObyT3BlbkFJivJ8OhSW43DNJlVp6w1D'

## Chat Models: GPT-3.5 Turbo and GPT-4

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
# output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo', max_tokens=100, temperature=0.5, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=['\n', ')
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo')
print(output.content)

Quantum mechanics is a branch of physics that describes the behavior of particles at the smallest scales, where traditional classical physics principles no longer apply and phenomena such as superposition and entanglement occur.


In [4]:
from langchain.schema import (
    SystemMessage, # corresponds to the OpenAI chat completion API
    AIMessage,     # Assistent message
    HumanMessage   # Human messages or prompt
)

messages = [    
    SystemMessage(content='You are a physicist and respond only in portuguese.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

A mecânica quântica é a teoria física que descreve o comportamento das partículas subatômicas em termos de estados superpostos, interferência e probabilidades.


## Caching LLM responses

### In-Memory cache

In [5]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [6]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a joke about quantum mechanics.'
llm.invoke(prompt)

CPU times: user 1.02 s, sys: 48.7 ms, total: 1.07 s
Wall time: 2.84 s


"\n\nWhy did Schrödinger's cat refuse to eat its food?\n\nBecause it was already in a superposition of being both hungry and full!"

After storing it in cache, the time to run the cell decreases to zero!!!

In [7]:
%%time
llm.invoke(prompt)

CPU times: user 1.75 ms, sys: 294 μs, total: 2.04 ms
Wall time: 2.09 ms


"\n\nWhy did Schrödinger's cat refuse to eat its food?\n\nBecause it was already in a superposition of being both hungry and full!"

### SQLite Caching

(to store in the SQLite caching)

In [8]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain_cache.db"))

prompt = 'Tell me a joke about quantum mechanics.'

# First request (not in cache, takes longer)
llm.invoke(prompt)

# Second request (in cache, takes less time)
llm.invoke(prompt)

"\n\nWhy did Schrödinger's cat refuse to come out of the box?\n\nBecause it was afraid of collapsing the waveform!"

### LLM Streaming

In [9]:
from langchain_openai import ChatOpenAI

# This is without streaming
llm = ChatOpenAI()
prompt = 'Tell me a joke about quantum mechanics.'
print(llm.invoke(prompt).content)

Why was Heisenberg such a terrible lover? 

Because when he had the position, he couldn't get the momentum, and when he had the momentum, he couldn't find the position!


In [10]:
# This is with streaming (piece by piece)
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

Why was the quantum physicist always calm during experiments? Because he had a lot of momentum!

### Prompt Templates

* Q&A 
* Phrase completions 
* Generating texts

In [11]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


template = ''' 
You are an experience physicist and you are explaining quantum mechanics to a student.
Write a short explanation about {topic}.
'''

prompt_template = PromptTemplate.from_template(template=template)
prompt = prompt_template.format(topic='tight binding model')
prompt


' \nYou are an experience physicist and you are explaining quantum mechanics to a student.\nWrite a short explanation about tight binding model.\n'

In [12]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

The tight binding model is a simplified approach used in quantum mechanics to describe the behavior of electrons in a solid material. In this model, we consider the electrons to be tightly bound to the atomic cores, and we focus on the interactions between neighboring atoms.

The model assumes that the electrons can only move within a certain range of energy levels, known as bands, which are determined by the interactions between neighboring atoms. These energy bands can be either filled with electrons or empty, depending on the material and its properties.

By studying the tight binding model, we can gain insights into the electronic structure of materials, such as their conductivity, magnetism, and optical properties. This model has been instrumental in understanding the behavior of electrons in solids and has been used to explain a wide range of phenomena in condensed matter physics.


### ChatPromptTemplates

In [30]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
     SystemMessage(content='You are a physicist and respond only in max 5 topics.'),
     HumanMessagePromptTemplate.from_template('Top {n} most spoken physics subject in {area}.'),
    ]
)

messages = chat_template.format_messages(n=5, area='Brazil')
print(messages)

[SystemMessage(content='You are a physicist and respond only in max 5 topics.'), HumanMessage(content='Top 5 most spoken physics subject in Brazil.')]


In [31]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

1. Particle Physics
2. Astrophysics
3. Condensed Matter Physics
4. Nuclear Physics
5. Quantum Mechanics
